In [1]:
## import datetime
from IPython.display import display
import itertools
import concurrent.futures
from matplotlib import pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import random
from scipy.optimize import least_squares, minimize
from scipy import linalg
import statistics as stats
import time
import warnings
warnings.filterwarnings('ignore')


# matrix helpers
def _yi(price_series):
    return [np.log(p) for p in price_series]

def _fi(tc, m, time_series):
    return [np.power((tc - t), m) if str(np.power((tc - t), m)) != 'nan' else 0 for t in time_series]
     
def _gi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.cos(w * np.log(tc - t)) if str(np.power((tc - t), m) * np.cos(w * np.log(tc - t))) != 'nan' else 0 for t in time_series]

def _hi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.sin(w * np.log(tc - t)) if str(np.power((tc - t), m) * np.sin(w * np.log(tc - t))) != 'nan' else 0 for t in time_series]

def _fi_pow_2(tc, m, time_series):
    return np.power(_fi(tc, m, time_series), 2)

def _gi_pow_2(tc, m, w, time_series):
    return np.power(_gi(tc, m, w, time_series), 2)

def _hi_pow_2(tc, m, w, time_series):
    return np.power(_hi(tc, m, w, time_series), 2)

def _figi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _gi(tc, m, w, time_series))

def _fihi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _hi(tc, m, w, time_series))

def _gihi(tc, m, w, time_series):
    return np.multiply(_gi(tc, m, w, time_series), _hi(tc, m, w, time_series))

def _yifi(tc, m, time_series, price_series):
    return np.multiply(_yi(price_series), _fi(tc, m, time_series))

def _yigi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _gi(tc, m, w, time_series))

def _yihi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _hi(tc, m, w, time_series))


In [2]:
# revised version of the LPPL without φ
# found on page 11 as equation (13)
def lppl(t, tc, m, w, a, b, c1, c2):
    return a + np.power(tc - t, m) * (b + ((c1 * np.cos(w * np.log(tc - t))) + (c2 * np.sin(w * np.log(tc - t)))))

# finds the least square difference
def func_restricted(x, *args):
    
    tc = x[0]
    m  = x[1]
    w  = x[2]
    
    data_series = args[0]
       
    lin_vals = matrix_equation(tc, m, w, data_series)

    a  = lin_vals[0] 
    b  = lin_vals[1]
    c1 = lin_vals[2] 
    c2 = lin_vals[3]

    delta = [lppl(t, tc, m, w, a, b, c1, c2) for t in data_series[0]]
    delta = np.subtract(delta, data_series[1])
    delta = np.power(delta, 2)
    return np.sum(delta)

# solve the matrix equation
def matrix_equation(tc, m, w, data_series):
    time_series = data_series[0]
    price_series = data_series[1]
    N  = len(price_series)
    
    #--------------------------------
    fi = sum(_fi(tc, m, time_series))
    gi = sum(_gi(tc, m, w, time_series))
    hi = sum(_hi(tc, m, w, time_series))

    #--------------------------------
    fi_pow_2 = sum(_fi_pow_2(tc, m, time_series))
    gi_pow_2 = sum(_gi_pow_2(tc, m, w, time_series))
    hi_pow_2= sum(_hi_pow_2(tc, m, w, time_series))

    #--------------------------------
    figi = sum(_figi(tc, m, w, time_series))
    fihi = sum(_fihi(tc, m, w, time_series))
    gihi = sum(_gihi(tc, m, w, time_series))

    #--------------------------------
    yi = sum(_yi(price_series))
    yifi = sum(_yifi(tc, m, time_series, price_series))
    yigi = sum(_yigi(tc, m, w, time_series, price_series))
    yihi = sum(_yihi(tc, m, w, time_series, price_series))
    
    #--------------------------------
    matrix_1 = np.matrix([
        [N,  fi,       gi,       hi      ],
        [fi, fi_pow_2, figi,     fihi    ],
        [gi, figi,     gi_pow_2, gihi    ],
        [hi, fihi,     gihi,     hi_pow_2]
    ])
    
    matrix_2 = np.matrix([
        [yi],
        [yifi],
        [yigi],
        [yihi]
    ])

    try:
        product = linalg.solve(matrix_1, matrix_2)
        return [i[0] for i in product]

    except Exception as e:
        print(e)
        
        
"""
upperbound = 126 # ~6 months (in trading days)
lowerbound = 21 # ~1 month (in trading days)
interval = 5
"""       
def compute_ds_lppls_confidence(args):
    
    df, symbol, upperbound, lowerbound, interval = args
    
    df2 = pd.DataFrame(df)
    
    ds_lppls = []
    
    number_of_fitting_windows = int((upperbound - lowerbound)/interval)
    
    for i in range(number_of_fitting_windows):

        tLen = upperbound-(i*interval)
        tradings_days_data = df.tail(tLen)
        time = np.linspace(0, tLen-1, tLen)
        price = np.array([tradings_days_data[i] for i in range(len(tradings_days_data))])
        data_series = np.array([time, price])

        MAX_SEARCHES = 5
        SEARCH_COUNT = 0
        
        # set limits for non-linear params
        bounds = [
            (tLen-(tLen*0.2), tLen+(tLen*0.2)),    # Critical Time + or - .2
            (0.1, 0.9),                            # m : 0.1 ≤ m ≤ 0.9
            (6, 13),                               # ω : 6 ≤ ω ≤ 13     
        ]

        # find bubbles
        while SEARCH_COUNT < MAX_SEARCHES:

            # randomly choose vals for non-linear params 
            non_lin_vals = [random.uniform(a[0], a[1]) for a in bounds]

            tc = non_lin_vals[0]
            m  = non_lin_vals[1] 
            w  = non_lin_vals[2]

            # params to pass to scipy.optimize
            seed = np.array([tc, m, w])

            # scipy optimize minimize
            try:
                # Nelder-Mead
                cofs = minimize(
                    args=(data_series, bounds), 
                    fun=func_restricted, 
                    method='Nelder-Mead', 
                    options={
                        'adaptive': True
                    },
                    x0=seed
                )
                
                if cofs.success:
#                     print('minimize ran succsessfully in {} search(es)'.format(SEARCH_COUNT+1))
                    # determine if it falls in range:

                    tc = cofs.x[0]
                    m =  cofs.x[1]
                    w =  cofs.x[2]
                    
                    # calculate the linear vals again...
                    lin_vals = matrix_equation(tc, m, w, data_series)

                    a  = lin_vals[0] 
                    b  = lin_vals[1]
                    c1 = lin_vals[2] 
                    c2 = lin_vals[3]
                    
                    # filtering conditions 
                    tc_in_range   = tLen-(tLen*0.05) < tc < tLen+(tLen*0.1)
                    m_in_range    = 0.01 < m < 1.2
                    w_in_range    = 2 < w < 25
                    n_oscillation = ((w/2)*np.log(abs((tc - (i*5))/(tLen)))) > 2.5
                    # for bubble end flag
                    damping_bef   = (m*abs(b))/(w*abs(c1+c2)) > 0.8 
                    # for bubble early warning 
                    damping_bew   = (m*abs(b))/(w*abs(c1+c2)) > 0.0 

                    if (tc_in_range and m_in_range and w_in_range and n_oscillation and damping_bef):
                        ds_lppls_confidence_bef = True
                    else: 
                        ds_lppls_confidence_bef = False
                        
                    if (tc_in_range and m_in_range and w_in_range and n_oscillation and damping_bew):
                        ds_lppls_confidence_bew = True
                    else: 
                        ds_lppls_confidence_bew = False   

                    ds_lppls.append({symbol: {
                        'ds_lppls_confidence_bef': ds_lppls_confidence_bef,
                        'ds_lppls_confidence_bew': ds_lppls_confidence_bew,
                        'cof': cofs.x,
                    }})
                    break

                else:
                    SEARCH_COUNT = SEARCH_COUNT + 1
#                     print('minimize failed to find a solution, trying again')
                    
            except Exception as e:
#                 print('minimize failed: {}'.format(e))
                SEARCH_COUNT = SEARCH_COUNT + 1
                
        if SEARCH_COUNT >= MAX_SEARCHES:
            # no solution found in 5 tries, so just add this and move one
#             print('minimize failed in allotted attempts (5)')
            ds_lppls.append({symbol: {
                'ds_lppls_confidence_bef': False,
                'ds_lppls_confidence_bew': False,
                'cof': None,
                'max_searches_exceeded': True
            }})
    
    # calculate the actual ds lppls confidence value for end flag and early warning
    true_count_bef = 0
    true_count_bew = 0
    total_count = len(ds_lppls)
    for i in ds_lppls:
        if i[symbol]['ds_lppls_confidence_bef'] == True:
            true_count_bef = true_count_bef + 1
        if i[symbol]['ds_lppls_confidence_bew'] == True:
            true_count_bew = true_count_bew + 1   
            
    ds_lppls_confidence_bef_val = true_count_bef/total_count
    ds_lppls_confidence_bew_val = true_count_bew/total_count
    
    # find the sign of the median of cumulative returns since the time t1
    df2['ret']  = (df2[symbol]/df2[symbol].shift(1))-1
    df2['cum_ret'] = df2['ret'].cumsum()
#     print(df2.head())
    median = stats.median(df2['cum_ret'].tolist())
#     print('median: {}'.format(median))
    median_sign = 1 if median > 0 else -1
#     print('median_sign: {}'.format(median_sign))
    
    return {
        'val_bef': ds_lppls_confidence_bef_val * median_sign,
        'val_bew': ds_lppls_confidence_bew_val * median_sign,
        'date': df.tail(1).index.tolist()[0],
        'price': df.tail(1).tolist()[0],
        'symbol': symbol,
    }

In [3]:
master_csv = 'https://boulderinvestmenttech.s3.amazonaws.com/iex_etl/data/agg/master.csv'
data = pd.read_csv(master_csv, index_col="date")
data.fillna(method='ffill')

master_data_len = len(data)
window_len = 126
# ground_to_cover = 1 # days
workers = 96 # 12 locked-up your macbook bro

# list of symbols from iex_etl/config.py 
SP500 = ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CDW', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'EVRG', 'ES', 'RE', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FLIR', 'FLS', 'FMC', 'F', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS', 'GWW', 'HAL', 'HBI', 'HOG', 'HIG', 'HAS', 'HCA', 'HCP', 'HP', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HFC', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'IR', 'INTC', 'ICE', 'IBM', 'INCY', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'JEC', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM', 'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KSS', 'KHC', 'KR', 'LB', 'LHX', 'LH', 'LRCX', 'LW', 'LEG', 'LDOS', 'LEN', 'LLY', 'LNC', 'LIN', 'LKQ', 'LMT', 'L', 'LOW', 'LYB', 'MTB', 'MAC', 'M', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MKC', 'MXIM', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MHK', 'TAP', 'MDLZ', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'MYL', 'NDAQ', 'NOV', 'NKTR', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NBL', 'JWN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'ORLY', 'OXY', 'OMC', 'OKE', 'ORCL', 'PCAR', 'PKG', 'PH', 'PAYX', 'PYPL', 'PNR', 'PBCT', 'PEP', 'PKI', 'PRGO', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTN', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'SHW', 'SPG', 'SWKS', 'SLG', 'SNA', 'SO', 'LUV', 'SPGI', 'SWK', 'SBUX', 'STT', 'SYK', 'STI', 'SIVB', 'SYMC', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL', 'FTI', 'TFX', 'TXN', 'TXT', 'TMO', 'TIF', 'TWTR', 'TJX', 'TSCO', 'TDG', 'TRV', 'TRIP', 'TSN', 'UDR', 'ULTA', 'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UTX', 'UHS', 'UNM', 'VFC', 'VLO', 'VAR', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VIAB', 'V', 'VNO', 'VMC', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WCG', 'WFC', 'WELL', 'WDC', 'WU', 'WRK', 'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XYL', 'YUM', 'ZBH', 'ZION', 'ZTS']

# SP500 = ['MMM', 'ABT', 'ABBV']
start = time.time()

pool = multiprocessing.Pool(processes=workers)
# compute the ds_lppls_confidence
# `data[symbol].iloc[-window_len:]` returns the tail of the data df of length `window_len`
result = pool.map(compute_ds_lppls_confidence, [(data[symbol].iloc[-window_len:], symbol, window_len, 21, 5) for symbol in SP500])
pool.close()
end = time.time()
duration_of_fit = end - start
print(duration_of_fit) 
print(result)
print('------------')

df_to_csv = pd.DataFrame(result)
df_to_csv.to_csv('~/indicators/2019/10/29.csv')

array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs 

array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matr

array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs 

array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not conta

Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs


...

array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array mu

Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is si

array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not conta

Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
array mu

Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
array must not contain infs or NaNs
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
Matrix is singular.
Matrix is singular.
array must not contain infs or NaNs
array mu

In [ ]:
pd.DataFrame(result)

In [ ]:
#plot the confidence indicators
fig, ax1 = plt.subplots(figsize=(25,12))

plt.xticks(rotation=90)

color = 'tab:blue'
ax1.set_xlabel('time (days)')
ax1.set_ylabel('price', color=color)
ax1.plot(t, price, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('val_bef', color=color)  # we already handled the x-label with ax1
ax2.plot(t, val_bef, color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax3.set_ylabel('val_bew', color=color)  # we already handled the x-label with ax1
ax3.plot(t, val_bew, color=color)
ax3.tick_params(axis='y', labelcolor=color)